In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import yaml
import pandas as pd
import numpy as np 
import seaborn as sns

In [3]:
from sklearn.linear_model import LogisticRegression
# importing library for support vector machine classifier
from sklearn.svm import SVC
# importing library for K-neares-neighbor classifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [4]:
##This packages are written by own in .py file inside dashboard
from dashboard import change_label
from dashboard.modeling import standardization, encode_dataframe
from dashboard import Visualization
from dashboard import DataInformation
from dashboard import show_result

In [5]:
with open("config.yaml", "r") as stream:
    cl = yaml.safe_load(stream)

In [6]:
train = cl['train']
feature = cl['feature']
test = cl['test']
test21 = cl['test21']
model_path = cl['model_path']
encoder_path = cl['encoder_path']
scaler_path = cl['scaler_path']

In [22]:
train_data=pd.read_csv(train,names=feature)

In [23]:
train_info = DataInformation(train_data)

In [24]:
train_data = train_info.drop_column("difficulty")

In [28]:
final_label_list = ['Dos','R2L','Probe','U2R']

In [29]:
for label in final_label_list:
    change_label(train_data, cl[label], label)

In [35]:
train_info_lc = DataInformation(train_data)

In [30]:
train_data['label'].unique()

array(['normal', 'Dos', 'R2L', 'Probe', 'U2R'], dtype=object)

In [31]:
train_without_label=train_data.drop('label',axis = 1)
train_without_label

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00
1,0,udp,other,SF,146,0,0,0,0,0,...,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00
2,0,tcp,private,S0,0,0,0,0,0,0,...,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00
3,0,tcp,http,SF,232,8153,0,0,0,0,...,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,tcp,private,S0,0,0,0,0,0,0,...,255,25,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00
125969,8,udp,private,SF,105,145,0,0,0,0,...,255,244,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00
125970,0,tcp,smtp,SF,2231,384,0,0,0,0,...,255,30,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00
125971,0,tcp,klogin,S0,0,0,0,0,0,0,...,255,8,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00


In [32]:
std_data = standardization(encode_dataframe(train_without_label.copy(), list(train_info.get_categorical_data_columns())[0:3],encoder_path), train_without_label.columns, scaler_path)                         
std_data.columns

/home/bhupin/akash_vai/venv/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bhupin/akash_vai/venv/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Index(['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
       'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate'],
      dtype='object')

In [39]:
label = encode_dataframe(pd.DataFrame(train_info_lc.get_data_label().copy()),['label'], encoder_path)


In [41]:
label['label'].unique()

array([4, 0, 2, 1, 3])

In [42]:
svm = SVC(kernel='linear',gamma='auto')
knn = knn=KNeighborsClassifier(n_neighbors=5)
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')

In [ ]:
show_result(std_data, label, knn, model_path)

/home/bhupin/akash_vai/venv/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
